## NLP Praktisches Notebook

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/nlpipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

---

### Thema 4: **Aktuelle Tools**


### ChatGPT

### Huggingface

In [32]:
from huggingface_hub import hf_hub_download

model_id = "google/flan-t5-base"
filenames = ["pytorch_model.bin","config.json","generation_config.json","special_tokens_map.json","spiece.model","tokenizer_config.json"]
for file in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id,
        filename = file,
    )
    print(downloaded_model_path)

/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/pytorch_model.bin
/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/generation_config.json
/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/special_tokens_map.json
/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/spiece.model
/Users/jonathan.bach/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/tokenizer_config.json


**Achtung das laden der Pipeline kann etwas dauern!**

Am besten Englisch verwenden

**e** drücken zum beenden!

In [33]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

generator = pipeline("text2text-generation", model=model, device=-1, tokenizer=tokenizer)

while True:
    question = input("Stelle deine Frage(e zum beenden): ")
    answer = generator(question)
    print(answer[0]["generated_text"])

    if question.lower() == "e":
        break

Device set to use cpu


a tiger
- The Associated Press - The Associated Press is a news organization that covers the
e


**Tiny Llama**

In [35]:
from huggingface_hub import hf_hub_download

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
filenames = [
    "model.safetensors",
    "config.json",
    "eval_results.json",
    "tokenizer_config.json",
    "tokenizer.json",
    "special_tokens_map.json",
    "tokenizer.model",
    "generation_config.json"
]

for file in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id,
        filename=file,
    )
    print(f"{file} -> {downloaded_model_path}")

model.safetensors -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/model.safetensors
config.json -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/config.json
eval_results.json -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/eval_results.json
tokenizer_config.json -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/tokenizer_config.json
tokenizer.json -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/tokenizer.json
special_tokens_map.json -> /Users/jonathan.bach/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snaps

In [37]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

while True:
    question = input("Stelle deine Frage (e zum Beenden): ")
    if question.lower() == "e":
        break

    response = generator(
        question,
        max_new_tokens=200,          # Begrenzung!
        do_sample=True,              # zufälligere Antworten
        temperature=0.7,             # Kreativität
        top_p=0.9                    # typische Sampling-Kombi
    )

    print(response[0]["generated_text"])

Device set to use cpu


Hello Who are you?


KeyboardInterrupt: 